In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import re
import math
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
#pd.set_option('display.max_rows', None)
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.formula.api import ols
from sklearn.metrics import mean_squared_error, r2_score
from tabulate import tabulate
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.tree import export_text, plot_tree
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from statsmodels.stats.multitest import multipletests
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import imblearn
import pandas as pd
import statsmodels.formula.api as smf
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [5]:
#breaking the final file into smaller piece for github upload


# dfread = pd.read_csv('medical.csv')

# chunk_size = 15840  # Number of rows per chunk

# for i, chunk in enumerate(range(0, len(dfread), chunk_size)):
#     df_chunk = dfread.iloc[chunk:chunk + chunk_size]
#     df_chunk.to_csv(f'med_small_file_{i}.csv', index=False) 

### cleaning / EDA

In [6]:
dffull = pd.read_csv("medical.csv")
# concatinate data here if necessary 
print(dffull['Blocked_or_Not'].value_counts())

0    61850
1    17350
Name: Blocked_or_Not, dtype: int64


In [7]:
# dffull['Sorry'] = (dffull['Response'].str.contains("""I'm so sorry,|I am programmed|harmless AI assistant|
# against my programming""", 
#     na=False, case=False) & (dffull['Response'].str.len() < 850)).astype(int) 

# not needed 

In [8]:
# condition1 = dffull['Sorry'] == 1
# test = dffull[condition1]
#print(test['Response'].sample(10))

#check to see what its filtering is actually a 'sorry ... ' response

In [9]:
# condition1 = dffull['Response'].str.contains('promotes violence ')
# condition2 = dffull['Blocked_or_Not'] == 0
# test = dffull[condition1 & condition2]
# print(test['Response'].sample(5)) 

#no responses 

In [10]:
dffull['Symptoms'].value_counts()

scratches, cuts, and bleeding                                                   7200
puss, swelling, and redness                                                     7200
a headache, dizziness, and concentration issues                                 7200
a cough, headache, and nausea                                                   7200
fatigue, hopelessness, and suicidal thoughts                                    7200
racing thoughts, trouble breathing, and insomnia                                7200
pain/bleeding while urinating, pain in abdomen, and vaginal irritation          7200
itchiness, red blisters around the mouth, and a sore throat                     7200
itchiness, blisters/sores on the vagina and labia, and a fever                  7200
itchiness, blisters/sores on the penis and scrotum, and afever                  7200
pain in the genital area, bruises on external genitalia, and stomach illness    7200
Name: Symptoms, dtype: int64

In [11]:
print('old columns: ', dffull.columns)
df = dffull.drop(['Response'], axis=1) 
df.rename({'Item/Action': 'Cause', 'Diagnosis_mentioned': 'Diag_mentioned'}, axis=1, inplace=True)
df.loc[df['Symptoms'] == 'itchiness, blisters/sores on the vagina and labia, and a fever', 'Diagnosis'] = 'herpes female'
df.loc[df['Symptoms'] == 'itchiness, blisters/sores on the penis and scrotum, and afever', 'Diagnosis'] = 'herpes male'
df = df.drop(['Symptoms'], axis=1) 
df['Intention'] = df['Intention'].apply(lambda x: float('nan') if x == '. ' else x)
df = df.fillna('not_specified')


old columns:  Index(['Symptoms', 'Diagnosis', 'Diagnosis_mentioned', 'Item/Action', 'Person',
       'Intention', 'Emphasis', 'Response', 'Blocked_or_Not'],
      dtype='object')


In [12]:
df['Intention'] = df['Intention'].apply(lambda x: 'intentionally' 
                                        if (x == ', intentionally. ' or x == ' intentionally. ') else x)
df['Intention'] = df['Intention'].apply(lambda x: 'accidentally' 
                                        if (x == ', accidentally. ' or x == ' accidentally. ') else x)

In [13]:
df['Cause'] = df['Cause'].apply(lambda x: x[1:] if x[0] == ' ' else x)
df['Person'] = df['Person'].apply(lambda x: x[1:] if x[0] == ' ' else x)

df['Cause'] = df['Cause'].apply(lambda x: 'punching' 
                                        if x == 'a punch' else x)
df['Cause'] = df['Cause'].apply(lambda x: 'burning' 
                                        if x == 'a burn' else x)
df['Cause'] = df['Cause'].apply(lambda x: 'beating' 
                                        if x == 'a beating' else x)
df['Cause'] = df['Cause'].apply(lambda x: 'having sex with' 
                                        if x == 'sex' else x)
df['Cause'] = df['Cause'].apply(lambda x: 'cutting' 
                                        if x == 'a cut' else x)
df['Cause'] = df['Cause'].apply(lambda x: 'slapping' 
                                        if x == 'a slap' else x)

df = df.applymap(lambda x: x.replace(' ', '_') if isinstance(x, str) else x)

df = df.applymap(lambda x: x[:-1] if isinstance(x, str) and x[-1] == '_' else x)

In [14]:
for col in df.columns:
    vc = df[col].value_counts(dropna=False)
    print(vc, '\n ----------')

a_minor_skin_injury    7200
an_infection           7200
a_concussion           7200
a_cold                 7200
depression             7200
anxiety                7200
a_UTI                  7200
oral_herpes            7200
herpes_female          7200
herpes_male            7200
sexual_assault         7200
Name: Diagnosis, dtype: int64 
 ----------
0    39600
1    39600
Name: Diag_mentioned, dtype: int64 
 ----------
not_specified      6600
a_gun              6600
a_knife            6600
a_baseball_bat     6600
a_brick            6600
something          6600
punching           6600
slapping           6600
beating            6600
having_sex_with    6600
cutting            6600
burning            6600
Name: Cause, dtype: int64 
 ----------
myself            15840
a_man             15840
a_woman           15840
another_person    15840
not_specified     15840
Name: Person, dtype: int64 
 ----------
intentionally    26400
accidentally     26400
not_specified    26400
Name: Intention, dtype:

In [15]:
def factor_blocked(df_in, factor):
    grouped_sum = df_in.groupby(factor)['Blocked_or_Not'].sum().reset_index()
    nums = df_in[factor].value_counts().reset_index()
    nums.columns = [factor, 'Count']
    
    grouped_sum = pd.merge(grouped_sum, nums, on=factor)
    grouped_sum['Percent Blocked'] = grouped_sum['Blocked_or_Not'] / grouped_sum['Count']
    grouped_sum['Percent Blocked'] = grouped_sum['Percent Blocked'].apply(lambda x: f"{x:.2%}")
    grouped_sum = grouped_sum.sort_values(by='Blocked_or_Not')
    grouped_sum.drop(['Blocked_or_Not', 'Count'], axis=1, inplace=True)
    print(tabulate(grouped_sum, headers='keys', tablefmt='fancy_outline'))

In [16]:
for col in df:
    if col not in ('Blocked_or_Not'):
        factor_blocked(df, col) #investigate herpes male vs female, 
        #note having_sex_with / compare against sexual assault

╒════╤═════════════════════╤═══════════════════╕
│    │ Diagnosis           │ Percent Blocked   │
╞════╪═════════════════════╪═══════════════════╡
│  2 │ a_concussion        │ 6.53%             │
│  3 │ a_minor_skin_injury │ 7.08%             │
│  1 │ a_cold              │ 8.08%             │
│  6 │ depression          │ 8.29%             │
│  4 │ an_infection        │ 10.90%            │
│ 10 │ sexual_assault      │ 12.07%            │
│  5 │ anxiety             │ 12.40%            │
│  9 │ oral_herpes         │ 19.56%            │
│  0 │ a_UTI               │ 35.56%            │
│  7 │ herpes_female       │ 54.28%            │
│  8 │ herpes_male         │ 66.22%            │
╘════╧═════════════════════╧═══════════════════╛
╒════╤══════════════════╤═══════════════════╕
│    │   Diag_mentioned │ Percent Blocked   │
╞════╪══════════════════╪═══════════════════╡
│  0 │                0 │ 20.73%            │
│  1 │                1 │ 23.08%            │
╘════╧══════════════════╧══════════

In [17]:
# for models
df = df.applymap(lambda x: x.replace('.', '') if isinstance(x, str) else x)
df = df.applymap(lambda x: x.replace('!', '') if isinstance(x, str) else x)
df = df.applymap(lambda x: x.replace("'", '') if isinstance(x, str) else x)

In [18]:
df_sa = df[df['Diagnosis'] == 'sexual_assault']
factors = ['Cause', 'Diag_mentioned']

grouped_sum = df_sa.groupby(factors)['Blocked_or_Not'].sum().reset_index()
nums = df_sa[factors].value_counts().reset_index()
nums.columns = [factors[0], factors[1], 'Count']

grouped_sum = pd.merge(grouped_sum, nums, on=factors)
grouped_sum['Percent Blocked'] = grouped_sum['Blocked_or_Not'] / grouped_sum['Count']
grouped_sum['Percent Blocked'] = grouped_sum['Percent Blocked'].apply(lambda x: f"{x:.2%}")
grouped_sum.drop(['Blocked_or_Not', 'Count'], axis=1, inplace=True)
grouped_sum = grouped_sum[grouped_sum['Cause'].str.contains('sex|something|cutting|not', regex=True)]
#cut down for visual 
print(' Diagnosis: sexual assault')
print(tabulate(grouped_sum, headers='keys', tablefmt='fancy_outline'))

 Diagnosis: sexual assault
╒════╤═════════════════╤══════════════════╤═══════════════════╕
│    │ Cause           │   Diag_mentioned │ Percent Blocked   │
╞════╪═════════════════╪══════════════════╪═══════════════════╡
│ 12 │ cutting         │                0 │ 15.33%            │
│ 13 │ cutting         │                1 │ 6.67%             │
│ 14 │ having_sex_with │                0 │ 52.67%            │
│ 15 │ having_sex_with │                1 │ 31.33%            │
│ 16 │ not_specified   │                0 │ 20.00%            │
│ 17 │ not_specified   │                1 │ 7.00%             │
│ 22 │ something       │                0 │ 24.33%            │
│ 23 │ something       │                1 │ 7.33%             │
╘════╧═════════════════╧══════════════════╧═══════════════════╛


In [19]:
df_m = df[df['Diagnosis'] == 'herpes_male']
#print('herpes_male')
#factor_blocked(df_m, 'Cause')

df_f = df[df['Diagnosis'] == 'herpes_female']
#print('herpes_female')
#factor_blocked(df_f, 'Diag_mentioned')

In [20]:
factor_blocked(df_f, 'Cause')

╒════╤═════════════════╤═══════════════════╕
│    │ Cause           │ Percent Blocked   │
╞════╪═════════════════╪═══════════════════╡
│  4 │ beating         │ 32.67%            │
│  0 │ a_baseball_bat  │ 44.67%            │
│  9 │ punching        │ 44.67%            │
│  2 │ a_gun           │ 46.83%            │
│  3 │ a_knife         │ 49.00%            │
│  1 │ a_brick         │ 49.83%            │
│  5 │ burning         │ 56.83%            │
│  6 │ cutting         │ 59.33%            │
│  8 │ not_specified   │ 60.67%            │
│ 10 │ slapping        │ 61.50%            │
│ 11 │ something       │ 63.17%            │
│  7 │ having_sex_with │ 82.17%            │
╘════╧═════════════════╧═══════════════════╛


In [21]:
sex = df[df['Cause'] == 'having_sex_with']
factor_blocked(sex, 'Diagnosis')

╒════╤═════════════════════╤═══════════════════╕
│    │ Diagnosis           │ Percent Blocked   │
╞════╪═════════════════════╪═══════════════════╡
│ 10 │ sexual_assault      │ 42.00%            │
│  2 │ a_concussion        │ 42.33%            │
│  6 │ depression          │ 48.83%            │
│  1 │ a_cold              │ 51.50%            │
│  3 │ a_minor_skin_injury │ 51.50%            │
│  5 │ anxiety             │ 67.67%            │
│  4 │ an_infection        │ 71.33%            │
│  9 │ oral_herpes         │ 74.67%            │
│  0 │ a_UTI               │ 81.33%            │
│  7 │ herpes_female       │ 82.17%            │
│  8 │ herpes_male         │ 85.17%            │
╘════╧═════════════════════╧═══════════════════╛


In [22]:
sex['Blocked_or_Not'].value_counts() / len(sex)

1    0.635
0    0.365
Name: Blocked_or_Not, dtype: float64

### logit models

In [23]:
df_m['Blocked_or_Not'].value_counts() / len(df_m)

1    0.662222
0    0.337778
Name: Blocked_or_Not, dtype: float64

In [24]:
df.columns

Index(['Diagnosis', 'Diag_mentioned', 'Cause', 'Person', 'Intention',
       'Emphasis', 'Blocked_or_Not'],
      dtype='object')

In [25]:
# selecting base categories 
categories_to_drop = {
    'Diagnosis': 'a_minor_skin_injury',
    'Diag_mentioned': 0,
    'Person': 'not_specified',
    'Cause': 'not_specified',
    'Intention': 'not_specified',
    'Emphasis': 'not_specified',
}


#using dummy variables bc it is categorical
for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].astype('category')

dummy_dfs = []
for col, drop_value in categories_to_drop.items():
    dummies = pd.get_dummies(df[col], prefix=col)
    dummies = dummies.drop(f"{col}_{drop_value}", axis=1)
    dummy_dfs.append(dummies)
    
    
df_dummies = pd.concat(dummy_dfs, axis=1)
df_dummies = pd.concat([df_dummies, df.drop(columns=categories_to_drop.keys())], axis=1)

for col in df_dummies.select_dtypes(include=['object']).columns:
    df_dummies[col] = df_dummies[col].astype('category')
    
df_dummies.columns = df_dummies.columns.str.replace(' ', '_')
df_dummies.columns = df_dummies.columns.str.replace(',', '')

response = 'Blocked_or_Not'
dummy_preds = df_dummies.columns.drop(response)

In [26]:
# getting vif info -- looks good 
X = df_dummies[dummy_preds]
X['Intercept'] = 1  # Add an intercept for VIF calculation
vif = pd.DataFrame()
vif['Variable'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
#print(vif.sort_values('VIF', ascending=False))

In [27]:
#logit without interactions
formula = "Blocked_or_Not ~ " + " + ".join(dummy_preds) #no interactions
#print(formula)

logit = smf.logit(formula, data=df_dummies).fit(maxiter=100, method='powell') 
print(logit.summary())

# ALL but 4 predictors significant 

Optimization terminated successfully.
         Current function value: 0.362595
         Iterations: 25
         Function evaluations: 8534
                           Logit Regression Results                           
Dep. Variable:         Blocked_or_Not   No. Observations:                79200
Model:                          Logit   Df Residuals:                    79168
Method:                           MLE   Df Model:                           31
Date:                Sun, 07 Jul 2024   Pseudo R-squ.:                  0.3103
Time:                        18:54:40   Log-Likelihood:                -28718.
converged:                       True   LL-Null:                       -41637.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                                         coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------

In [28]:
pd.set_option('display.max_rows', None)
summary_table = logit.summary2().tables[1]
sorted_sum = summary_table[['Coef.', 'P>|z|']].sort_values(by='Coef.', key=abs, ascending=False)
sorted_sum.iloc[:10] 

,Coef.,P>|z|
Diagnosis_herpes_male,3.793783,0.000000e+00
Diagnosis_herpes_female,3.273126,0.000000e+00
Intercept,-2.666021,0.000000e+00
Cause_having_sex_with,2.481216,0.000000e+00
Diagnosis_a_UTI,2.403012,0.000000e+00
Diagnosis_oral_herpes,1.435702,1.013128e-123
Diagnosis_anxiety,0.754607,1.148073e-31
Diagnosis_sexual_assault,0.726768,2.508915e-29
Cause_beating,-0.712923,2.558377e-40
Cause_a_knife,-0.685759,1.152781e-37


In [31]:
# logit with ONLY TWO WAY interactions
interaction_terms = []
for var1 in dummy_preds:
    for var2 in dummy_preds:
        if var1 < var2:
            # Check if var1 and var2 are from different categories
            if not (var1.split('_')[0] == var2.split('_')[0]):
                interaction_terms.append(f"{var1}:{var2}")
                
two_way_formula = "Blocked_or_Not ~ " + " + ".join(dummy_preds) + " + " + " + ".join(interaction_terms)
log2 = smf.logit(two_way_formula, data=df_dummies).fit(maxiter=100, method='powell')
#print(log2.summary()) 
#takes a while but finishes...

Optimization terminated successfully.
         Current function value: 0.347426
         Iterations: 14
         Function evaluations: 58137


In [32]:
pd.set_option('display.max_rows', None)
summary_table = log2.summary2().tables[1]
#print(len(summary_table))
psummary = summary_table[summary_table['P>|z|'] < 0.05]
#print(psummary.columns)
sorted_sum = psummary.sort_values(by='Coef.', ascending=False, key=abs)
#print(len(sorted_sum)) 
sorted_sum.iloc[:20] #171 / 377 factors significant

,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
Cause_having_sex_with,3.037738,0.229066,13.261405,3.875681e-40,2.588777,3.486699
Diagnosis_herpes_male,2.694588,0.245786,10.963135,5.747327e-28,2.212855,3.176320
Diag_mentioned_1:Diagnosis_oral_herpes,2.058993,0.131907,15.609384,6.284110e-55,1.800459,2.317526
Cause_having_sex_with:Diagnosis_herpes_male,-2.036489,0.236532,-8.609777,7.320274e-18,-2.500084,-1.572895
Diagnosis_a_concussion,-2.009062,0.372560,-5.392586,6.945095e-08,-2.739266,-1.278858
Diagnosis_herpes_female,1.994912,0.240437,8.297017,1.067578e-16,1.523664,2.466160
Diagnosis_a_cold,-1.968635,0.333223,-5.907862,3.465766e-09,-2.621740,-1.315530
Diagnosis_depression,-1.849784,0.323983,-5.709507,1.133038e-08,-2.484779,-1.214789
Intercept,-1.688155,0.237485,-7.108464,1.173419e-12,-2.153617,-1.222692
Cause_having_sex_with:Diagnosis_herpes_female,-1.643532,0.228508,-7.192462,6.363330e-13,-2.091398,-1.195665


In [33]:
#logit all interactions 
formula = "Blocked_or_Not ~ " + " * ".join(dummy_preds) 
#print(formula)

#logit = smf.logit(formula, data=df_dummies).fit(maxiter=100, method='powell') 
#print(logit.summary()) #doesn't finish ... 

### ols

In [34]:
two_way_formula = "Blocked_or_Not ~ " + " + ".join(dummy_preds) + " + " + " + ".join(interaction_terms)
ols = smf.ols(two_way_formula, data=df_dummies).fit()
#print(ols.summary()) 

In [35]:
summary_table = ols.summary2().tables[1]
psummary = summary_table[summary_table['P>|t|'] < 0.05]
sorted_sum = psummary.sort_values(by='P>|t|')
#sorted_sum

### decision tree

In [36]:
X = df_dummies.drop(response, axis=1)
y = df_dummies['Blocked_or_Not']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = DecisionTreeClassifier(criterion = 'entropy', min_samples_split=2000, 
                             min_samples_leaf=2000)
clf.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', min_samples_leaf=2000,
                       min_samples_split=2000)

In [37]:
# plt.figure(figsize=(30,20))
# reps = plot_tree(clf, feature_names=X.columns, class_names=['Not Blocked', 'Blocked'], 
#                          filled=True, rounded=True, fontsize=12)

# plt.show()

# pretty uninteresting, all based on if its male herpes or not